This is the Python code of Example 3.13 in the book:

FIBER-REINFORCED COMPOSITES
Materials, Manufacturing, and Design

by: P.K. Mallick (2008) by Taylor & Francis Group, LLC

This code:
(C) Joris Remmers (2013-2019)


## Example 3.13

Calculate lamina stresses at the midplane of each lamina in the [+45/-45] laminate in Example 3.7 due to $N_{xx} = 100 kN/m$.

## Solution

Import the correct functions from the composite module and required mathermatical operators:

In [1]:
from composite    import TransverseIsotropic,mixMaterials,Laminate,stressTransformation
from numpy        import array,dot,zeros
from numpy.linalg import inv

As demonstrated in Example 3.6 and 3.7, the ply properties and laminate can be obtained as: 

In [2]:
carbon = TransverseIsotropic( 220e9,0.2,91.7e9)
epoxy  = TransverseIsotropic( 3.6e9,0.35,1.33e9)

compmat = mixMaterials( carbon , epoxy , 0.6 )

lam = Laminate()

lam.addMaterial( 'composite' , compmat )

lam.addLayer( 'composite' , -45 , 6e-3 )
lam.addLayer( 'composite' , 45 , 6e-3 )

print(lam)

  Laminate properties
  -----------------------------------------------------------
  layer   thick orient.  material
  -----------------------------------------------------------
      0   0.006    -45   composite
      1   0.006     45   composite



To be able to calculate the midplain strains and curvature, the inverse laminate stiffness matrices $[A_1]$, $[B_1]$, $[C_1]$, and $[D_1]$ are required. First, the $[A]$, $[B]$ and $[D]$ matrixes are computed, and then the corresponding inverse matrice.

In [3]:
A = lam.getA()
B = lam.getB()
D = lam.getD()

A1,B1,C1,D1 = lam.getInverseMatrices()

print("The matrix A1:\n",A1,"\n")
print("The matrix B1:\n",B1,"\n")
print("The matrix C1:\n",C1,"\n")
print("The matrix D1:\n",D1,"\n")

The matrix A1:
 [[ 7.73727620e-09 -5.06668026e-09  0.00000000e+00]
 [-5.06668026e-09  7.73727620e-09  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  5.69566724e-09]] 

The matrix B1:
 [[-0.0000000e+00 -0.0000000e+00 -6.0459327e-07]
 [-0.0000000e+00 -0.0000000e+00 -6.0459327e-07]
 [-6.0459327e-07 -6.0459327e-07 -0.0000000e+00]] 

The matrix C1:
 [[-0.0000000e+00 -0.0000000e+00 -6.0459327e-07]
 [-0.0000000e+00 -0.0000000e+00 -6.0459327e-07]
 [-6.0459327e-07 -6.0459327e-07 -0.0000000e+00]] 

The matrix D1:
 [[ 0.00064477 -0.00042222  0.        ]
 [-0.00042222  0.00064477  0.        ]
 [ 0.          0.          0.00047464]] 



Now the midplain strains and curvature can be calculated: 

In [4]:
N = zeros(3)
N[0] = 1e5

eps0 = dot(A1,N)

print("The midplane strains are:",eps0)

kappa = dot(C1,N)

print("The curvatures are:",kappa)


The midplane strains are: [ 0.00077373 -0.00050667  0.        ]
The curvatures are: [ 0.          0.         -0.06045933]


Using these midplain strains and curvature, the stresses and strains in the individual layers can be calculated as:

In [5]:
epsb = eps0-3e-3*kappa
epst = eps0+3e-3*kappa

print ("\nThe strains in the bottom layer : \n",epsb)
print ("\nThe strains in the top layer    : \n",epst)

stressb = dot(lam.getQbar(0),epsb)
stresst = dot(lam.getQbar(1),epst)

print ("\nThe stresses in the bottom layer : \n",stressb)
print ("\nThe stresses in the top layer    : \n",stresst)


The strains in the bottom layer : 
 [ 0.00077373 -0.00050667  0.00018138]

The strains in the top layer    : 
 [ 0.00077373 -0.00050667 -0.00018138]

The stresses in the bottom layer : 
 [ 8.33333333e+06  9.31322575e-10 -2.08995545e+06]

The stresses in the top layer    : 
 [8.33333333e+06 9.31322575e-10 2.08995545e+06]


The stresses can be expressed in the material frame of reference (12) by transformation:

In [7]:
print("\nThe stress in the bottom layer : \n",stressTransformation(stressb,-45.))
print("\nThe stress in the top layer    : \n",stressTransformation(stresst,45.))

-0.7853981633974483

The stress in the bottom layer : 
 [6256622.12152551 2076711.21180782 4166666.66666666]
0.7853981633974483

The stress in the top layer    : 
 [ 6256622.12152551  2076711.21180782 -4166666.66666666]
